# Train Model

In [1]:
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

from util import *


C:\Users\Lenovo\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Lenovo\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Lenovo\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Lenovo\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [2]:
print("Version: ", tf.__version__)
# tf.compat.v1.enable_eager_execution() # Enable eager execution
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.experimental.list_physical_devices('GPU') else "NOT AVAILABLE")
# print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  1.14.0
Eager mode:  False
GPU is NOT AVAILABLE


## File paths

In [3]:
HOME_DIR = pathlib.Path.cwd()

# feature_path = HOME_DIR / 'data' / 'processed'/ 'IEMOCAP' / 'extracted_feature.pk'
feature_path = 'D:/extracted_features.pk'

label_path = HOME_DIR / 'data' / 'processed' / 'IEMOCAP' / 'FC_label.txt'
processed_id_path = HOME_DIR / 'data' / 'processed' / 'IEMOCAP' / 'processed_ids.txt'
dataset_path = HOME_DIR / 'data' / 'raw'


## Training plots

In [4]:
def acc_plot(history, fold, save_path):
    '''
    Plot training accuracy graph.
    '''

    epochs = range(len(history.history['acc']))

    plt.plot(epochs, history.history['acc'], 'r', label='Training accuracy')
    plt.plot(epochs, history.history['val_acc'], 'b', label='Validation accuracy')
    plt.title(f'Training and validation accuracy (Text) - Fold#{fold}')
    plt.legend(loc=0)
    plt.figure()

    plt.savefig(save_path + f'/acc_fold_#{fold}.png')
    plt.show()

In [5]:
def loss_plot(history, fold, save_path):
    '''
    Plot training loss graph.
    '''

    epochs = range(len(history.history['loss']))

    plt.plot(epochs, history.history['loss'], 'r', label='Training loss')
    plt.plot(epochs, history.history['val_loss'], 'b', label='Validation loss')
    plt.title(f'Training and validation loss (Text) - Fold#{fold}')
    plt.legend(loc=0)
    plt.figure()

    plt.savefig(save_path + f'/loss_fold_#{fold}.png')
    plt.show()

In [38]:
def shuffle_dataset(features, labels):
    '''
    Shuffle features and labels in unison.
    '''
    shuffler = np.random.permutation(labels.index)

    features_shuffled, labels_shuffled = features[shuffler], labels.reindex(shuffler)

    return features_shuffled, labels_shuffled


## Load and prepare extracted data

In [7]:
# Load extracted feature data
features = np.load('D:/processed_features.npz')
features = features['arr_0']


In [36]:
# Load label dataframe
label = pd.read_pickle('D:/label_dataset.pkl')


In [39]:
features, label = shuffle_dataset(features, label)


In [43]:
print(f'Features shape --> {features.shape},\t Label shape --> {label.shape}')

Features shape --> (5531, 3409, 65),	 Label shape --> (5531, 1)


In [46]:
feature_train = features[:5000]
label_train = label[:5000]

feature_test = features[5000:]
label_test = label[5000:]

print(f'X-Train length --> {len(feature_train)}, Y-Train length --> {len(label_train)}')
print(f'X-Test length --> {len(feature_test)}, Y-Test length --> {len(label_test)}')

# data_dict = {'features':feature_train, 'label':label_train}
# dataset = pd.DataFrame.from_dict(data_dict)


X-Train length --> 5000, Y-Train length --> 5000
X-Test length --> 531, Y-Test length --> 531


## One-hot encode label

In [61]:
enc = OneHotEncoder(sparse=False)
label_encode = enc.fit_transform(label_train[['label']])


In [62]:
enc.categories_

[array(['ang', 'hap', 'neu', 'sad'], dtype=object)]

## Build model

In [63]:
# K-fold configuration
num_folds = 10

# Model configuration
batch_size = 64
loss_function = categorical_crossentropy
no_classes = 4
no_epochs = 30
grad_clip_value = 5.0
learn_rate = 1e-4
verbosity = 1
optimizer = Adam(learning_rate=learn_rate, clipnorm=grad_clip_value)

In [64]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=False)
fold = 1

acc_per_fold = []
loss_per_fold = []

In [ ]:
# Train model
for train_index, val_index in kfold.split(label_encode):

    X_train = feature_train[train_index]
    X_val = feature_train[val_index]

    y_train = label_encode[train_index]
    y_val = label_encode[val_index]


    model = tf.keras.Sequential([
                                tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, 
                                                                                   return_sequences=True,
                                                                                   activation= 'relu',
                                                                                   dropout=0.4, 
                                                                                   kernel_regularizer=regularizers.l2(1e-5)),
                                                              input_shape=(5531, 3409, 65)),
                                tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, 
                                                                                   activation= 'relu', 
                                                                                   dropout=0.5, 
                                                                                   kernel_regularizer=regularizers.l2(1e-5))),
                                # tf.keras.layers.Average(),
                                tf.keras.layers.Dense(128, activation='relu'),
                                tf.keras.layers.Dense(no_classes, activation='softmax')
    ])

    model.compile(loss=loss_function,
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    # Provide training information
    print('------------------------------------------------------------------------')
    print(f'Training for fold #{fold} ...')

    history = model.fit(X_train,
                        y_train,
                        epochs=no_epochs,
                        batch_size=batch_size,
                        verbose=verbosity,
                        validation_data=(X_val, y_val))
    
    acc_plot(history, fold, save_path='/content/drive/MyDrive/Colab Notebooks/My Project - SER/fold_result_plot/text')
    loss_plot(history, fold, save_path='/content/drive/MyDrive/Colab Notebooks/My Project - SER/fold_result_plot/text')
    
    loss, acc = model.evaluate(X_val, y_val, verbose=0)

    print(f'Saving Model for fold #{fold}...')
    model.save(f'/content/drive/MyDrive/Colab Notebooks/My Project - SER/text_fold_models/fold_{fold}_model.h5')

    # loss, acc = history.history['loss'][-1], history.history['acc'][-1]

    print(f'Score for fold #{fold}: {model.metrics_names[0]} of {loss}; {model.metrics_names[1]} of {acc * 100}%')
    acc_per_fold.append(acc * 100)
    loss_per_fold.append(loss)

    fold += 1 # Increase fold number


In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for index, acc in enumerate(acc_per_fold):
  print('------------------------------------------------------------------------')
  print(f'> Fold {index+1} - Loss: {loss_per_fold[index]} - Accuracy: {acc_per_fold[index]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')